## You can also run the notebook in [COLAB](https://colab.research.google.com/github/deepmipt/DeepPavlov/blob/master/examples/gobot_tutorial.ipynb).

In [ ]:
!pip install deeppavlov

## Intro

The tutor is focused on building a goal-oriented dialogue system.

An example of the final model served as a telegram bot is:

![gobot_example.png](img/gobot_example.png)

## Data Preparation

The tutor's dialogue system will be on the domain of restaurant booking. [Dialogue State Tracking Challenge 2 (DSTC-2)](http://camdial.org/~mh521/dstc/) dataset provides dialogues of a human talking to a booking system labelled with slots and dialogue actions. The labels are will be used for training a dialogue policy network.

See below a small chunk of the data. 

In [1]:
from deeppavlov.dataset_readers.dstc2_reader import SimpleDSTC2DatasetReader

data = SimpleDSTC2DatasetReader().read('my_data')

2019-09-02 18:02:38.261 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 269: [downloading data from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data]
2019-09-02 18:02:39.298 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/simple_dstc2.tar.gz to my_data/simple_dstc2.tar.gz
100%|██████████| 497k/497k [00:00<00:00, 43.2MB/s]
2019-09-02 18:02:39.314 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_data/simple_dstc2.tar.gz archive into my_data
2019-09-02 18:02:39.343 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 289: [loading dialogs from my_data/simple-dstc2-trn.json]
2019-09-02 18:02:39.414 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 289: [loading dialogs from my_data/simple-dstc2-val.json]
2019-09-02 18:02:39.465 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 289: [loading di

In [2]:
!ls my_data

simple-dstc2-templates.txt  simple-dstc2-tst.json
simple-dstc2-trn.json	    simple-dstc2-val.json


The training/validation/test data is stored in json files (`simple-dstc2-trn.json`, `simple-dstc2-val.json` and `simple-dstc2-tst.json`):

In [3]:
!head -n 101 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },
    {
      "speaker": 1,
      "text": "any",
      "slots": [
        [
          "this",
          "dontcare"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What part of town do you have in mind?",
      "slots": [],
      "act": "request_area"
    },
    {
      "speaker": 1,
      "text": "south",
      "slots": [
        [
          "area",
          "south"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "api_call area=\"south\" food

In [4]:
from deeppavlov.dataset_iterators.dialog_iterator import DialogDatasetIterator

iterator = DialogDatasetIterator(data)

You can now iterate over batches of preprocessed DSTC-2 dialogs:

In [5]:
from pprint import pprint

for dialog in iterator.gen_batches(batch_size=1, data_type='train'):
    turns_x, turns_y = dialog
    
    print("User utterances:\n----------------\n")
    pprint(turns_x[0], indent=4)
    print("\nSystem responses:\n-----------------\n")
    pprint(turns_y[0], indent=4)
    
    break

User utterances:
----------------

[   {'prev_resp_act': None, 'text': ''},
    {   'prev_resp_act': 'welcomemsg',
        'text': 'a moderately priced restaurant in the south part of town'},
    {   'db_result': {   'addr': '152 - 154 hills road',
                         'area': 'south',
                         'food': 'modern european',
                         'name': 'restaurant alimentum',
                         'phone': '01223 413000',
                         'postcode': 'c.b 2, 8 p.b',
                         'pricerange': 'moderate'},
        'prev_resp_act': 'api_call',
        'text': 'a moderately priced restaurant in the south part of town'},
    {   'prev_resp_act': 'inform_area+inform_pricerange+offer_name',
        'text': 'what is the address'},
    {'prev_resp_act': 'inform_addr+offer_name', 'text': 'thank you good bye'}]

System responses:
-----------------

[   {   'act': 'welcomemsg',
        'text': 'Hello, welcome to the Cambridge restaurant system. You can 

## 1. Build database of items

&nbsp;
![gobot_database.png](img/gobot_database.png)
&nbsp;

For a valid goal-oriented bot there should be a `database` of relevant items. In the case of restaurant booking it will contain all available restaurants and their info.

    >> database([{'pricerange': 'cheap', 'area': 'south'}])
    
    Out[1]: 
        [[{'name': 'the lucky star',
           'food': 'chinese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way cherry hinton',
           'phone': '01223 244277',
           'postcode': 'c.b 1, 7 d.y'},
          {'name': 'nandos',
           'food': 'portuguese',
           'pricerange': 'cheap',
           'area': 'south',
           'addr': 'cambridge leisure park clifton way',
           'phone': '01223 327908',
           'postcode': 'c.b 1, 7 d.y'}]]
           
The dialogues in the training dataset should contain a `"db_result"` dictionary key. It is required for turns where system performs a special type of external action: an api call to the database of items. `"db_result"` should contain the result of the api call:

In [6]:
!head -n 78 my_data/simple-dstc2-trn.json | tail +51

    {
      "speaker": 2,
      "text": "api_call area=\"south\" food=\"dontcare\" pricerange=\"cheap\"",
      "db_result": {
        "food": "chinese",
        "pricerange": "cheap",
        "area": "south",
        "addr": "cambridge leisure park clifton way cherry hinton",
        "phone": "01223 244277",
        "postcode": "c.b 1, 7 d.y",
        "name": "the lucky star"
      },
      "slots": [
        [
          "area",
          "south"
        ],
        [
          "pricerange",
          "cheap"
        ],
        [
          "food",
          "dontcare"
        ]
      ],
      "act": "api_call"
    },


In [7]:
from deeppavlov.core.data.sqlite_database import Sqlite3Database

database = Sqlite3Database(primary_keys=["name"],
                           save_path="my_bot/db.sqlite")

2019-09-02 18:02:46.620 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-02 18:02:46.621 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 70: Initializing empty database on /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.


Set `primary_keys` to a list of slot names that have unique values for different items (common SQL term). For the case of DSTC-2, the primary slot is restaurant name.

Let's find all `"db_result"` api call results and add it to our database of restaurants:

In [8]:
db_results = []

for dialog in iterator.gen_batches(batch_size=1, data_type='all'):
    turns_x, turns_y = dialog
    db_results.extend(x['db_result'] for x in turns_x[0] if x.get('db_result'))

print(f"Adding {len(db_results)} items.")
if db_results:
    database.fit(db_results)

2019-09-02 18:02:47.907 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 145: Created table with keys {'pricerange': 'text', 'name': 'text', 'area': 'text', 'food': 'text', 'postcode': 'text', 'phone': 'text', 'addr': 'text'}.


Adding 3016 items.


We can now play with the database and make requests to it:

In [9]:
database([{'pricerange': 'cheap', 'area': 'south'}])

[[{'pricerange': 'cheap',
   'name': 'the lucky star',
   'area': 'south',
   'food': 'chinese',
   'postcode': 'c.b 1, 7 d.y',
   'phone': '01223 244277',
   'addr': 'cambridge leisure park clifton way cherry hinton'},
  {'pricerange': 'cheap',
   'name': 'nandos',
   'area': 'south',
   'food': 'portuguese',
   'postcode': 'c.b 1, 7 d.y',
   'phone': '01223 327908',
   'addr': 'cambridge leisure park clifton way'}]]

In [10]:
!ls my_bot

db.sqlite


## 2. Build Slot Filler

&nbsp;
![gobot_slotfiller.png](img/gobot_slotfiller.png)
&nbsp;

Slot Filler is component that inputs text and outputs dictionary of slot names and their values:

    >> slot_filler(['I would like some chineese food'])
    
    Out[1]: [{'food': 'chinese'}]

To implement a slot filler you need to provide
    
 - **slot types**
 - all possible **slot values**
 - optionally, it will be good to provide examples of mentions for every value of each slot
 
The data should be in `slot_vals.json` file with the following format:

    {
        'food': {
            'chinese': ['chinese', 'chineese', 'chines'],
            'french': ['french', 'freench'],
            'dontcare': ['any food', 'any type of food']
        }
    }
                

Let's use a simple non-trainable slot filler that relies on levenshtein distance:

In [11]:
from deeppavlov.download import download_decompress

download_decompress(url='http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz',
                    download_path='my_bot/slotfill')

2019-09-02 18:02:54.630 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/deeppavlov_data/dstc_slot_vals.tar.gz to my_bot/slotfill/dstc_slot_vals.tar.gz
100%|██████████| 1.62k/1.62k [00:00<00:00, 1.96MB/s]
2019-09-02 18:02:54.638 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting my_bot/slotfill/dstc_slot_vals.tar.gz archive into my_bot/slotfill


In [12]:
!ls my_bot/slotfill

dstc_slot_vals.json


In [13]:
!head -n 10 my_bot/slotfill/dstc_slot_vals.json

{
    "food": {
        "caribbean": [
            "carraibean",
            "carribean",
            "caribbean"
        ],
        "kosher": [
            "kosher"
        ],


Let's check performance of our slot filler on DSTC-2 dataset:

In [14]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

slotfill_raw_config = read_json(configs.ner.slotfill_dstc2_raw)

We take [original DSTC2 slot-filling config](https://github.com/deepmipt/DeepPavlov/blob/master/deeppavlov/configs/ner/slotfill_dstc2_raw.json) and change variables determining data paths:

In [15]:
slotfill_raw_config['metadata']['variables']['DATA_PATH'] = 'my_data'
slotfill_raw_config['metadata']['variables']['SLOT_VALS_PATH'] = 'my_bot/slotfill/dstc_slot_vals.json'

In [16]:
from deeppavlov import evaluate_model

slotfill_raw = evaluate_model(slotfill_raw_config);

2019-09-02 18:03:15.686 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 95: [downloading data from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to /home/vimary/code-projects/Pilot/examples/my_data]
2019-09-02 18:03:15.703 INFO in 'deeppavlov.core.data.utils'['utils'] at line 63: Downloading from http://files.deeppavlov.ai/datasets/dstc2_v2.tar.gz to /home/vimary/code-projects/Pilot/examples/my_data/dstc2_v2.tar.gz
100%|██████████| 506k/506k [00:00<00:00, 49.1MB/s]
2019-09-02 18:03:15.715 INFO in 'deeppavlov.core.data.utils'['utils'] at line 201: Extracting /home/vimary/code-projects/Pilot/examples/my_data/dstc2_v2.tar.gz archive into /home/vimary/code-projects/Pilot/examples/my_data
2019-09-02 18:03:15.736 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-09-02 18:03:15.879 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2

{"valid": {"eval_examples_count": 1444, "metrics": {"slots_accuracy": 0.9307}, "time_spent": "0:00:28"}}
{"test": {"eval_examples_count": 1386, "metrics": {"slots_accuracy": 0.9481}, "time_spent": "0:00:26"}}


We've got slot accuracy of **93% on valid** set and **94% on test** set.

Let's interact with it:

In [17]:
from deeppavlov import build_model

slotfill_raw = build_model(slotfill_raw_config)

In [18]:
slotfill_raw(['i want cheap chinee food'])

[{'food': 'chinese', 'pricerange': 'cheap'}]

Saving slotfill config file to disk (we will require it's path later):

In [19]:
import json

json.dump(slotfill_raw_config, open('my_bot/slotfill_raw_config.json', 'wt'))

In [20]:
!ls my_bot

db.sqlite  slotfill  slotfill_raw_config.json


## 3. Train bot

Let's assemble all modules together and train the final module: dialogue policy network.

&nbsp;
![gobot_policy.png](img/gobot_policy.png)
&nbsp;

Policy network decides which action the system should take on each turn of a dialogue: should it say goodbye, request user's location or make api call to a database.

The policy network is a recurrent neural network (recurrent over utterances represented as bags of words) and a dense layer with softmax function on top. The network classifies user utterance into one of predefined system actions.

&nbsp;
![gobot_templates.png](img/gobot_templates.png)
&nbsp;

All actions available for the system should be listed in a `simple-dstc2-templates.txt` file. Each action should be associated with a string of the corresponding system response.

Templates should be in the format `<act>TAB<template>`, where `<act>` is a dialogue action and `<template>` is the corresponding response. Response text might contain slot type names, where every `#slot_type` will be filled with the slot value from a dialogue state.

In [21]:
!head -n 10 my_data/simple-dstc2-templates.txt

api_call	api_call area="#area" food="#food" pricerange="#pricerange"
bye	You are welcome!
canthear	Sorry, I can't hear you.
canthelp_area	I'm sorry but there is no #area american restaurant in the #area of town.
canthelp_area_food	Sorry there is no #food restaurant in the #area of town.
canthelp_area_food_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #food food.
canthelp_area_pricerange	Sorry there is no #pricerange restaurant in the #area of town serving #area american food.
canthelp_food	I am sorry but there is no #food restaurant that matches your request.
canthelp_food_pricerange	Sorry there is no #food restaurant in the #pricerange price range.
confirm-domain	You are looking for a restaurant is that right?


So, actions are actually classes we classify over. And `simple-dstc2-templates.txt` contains the set of classes.

To train the dialogue policy network for classification task you need action label for each system utterance in training dialogues. The DSTC-2 contains `"act"` dictionary key that contains action associated with current response.

The cell below provides an example of training data for the policy network.

In [23]:
!head -n 24 my_data/simple-dstc2-trn.json

[
  [
    {
      "speaker": 2,
      "text": "Hello, welcome to the Cambridge restaurant system. You can ask for restaurants by area, price range or food type. How may I help you?",
      "slots": [],
      "act": "welcomemsg"
    },
    {
      "speaker": 1,
      "text": "cheap restaurant",
      "slots": [
        [
          "pricerange",
          "cheap"
        ]
      ]
    },
    {
      "speaker": 2,
      "text": "What kind of food would you like?",
      "slots": [],
      "act": "request_food"
    },


Let's construct the final pipeline of a dialogue system:

In [24]:
from deeppavlov import configs
from deeppavlov.core.common.file import read_json

gobot_config = read_json(configs.go_bot.gobot_dstc2)

**Configure** bot to use **templates**:

In [25]:
gobot_config['chainer']['pipe'][-1]['template_type'] = 'DefaultTemplate'
gobot_config['chainer']['pipe'][-1]['template_path'] = 'my_data/simple-dstc2-templates.txt'

**Configure** bot to use our built **database**:

In [26]:
gobot_config['chainer']['pipe'][-1]['database'] = {
    'class_name': 'sqlite_database',
    'primary_keys': ["name"],
    'save_path': 'my_bot/db.sqlite'
}

**Configure** bot to use levenshtein distance based **slot filler**:

In [27]:
gobot_config['chainer']['pipe'][-1]['slot_filler']['config_path'] = 'my_bot/slotfill_raw_config.json'

gobot_config['chainer']['pipe'][-1]['tracker']['slot_names'] = ['pricerange', 'this', 'area', 'food']

You can use a simple **bag-of-words as embedder** (by default):

In [28]:
gobot_config['chainer']['pipe'][-1]['embedder'] = None

Specify train/valid/test **data path** and **path to save** the final bot model:

In [29]:
gobot_config['metadata']['variables']['DATA_PATH'] = 'my_data'

gobot_config['metadata']['variables']['MODEL_PATH'] = 'my_bot'

The whole dialogue system pipeline looks like this:
    
![gobot_pipeline.png](img/gobot_pipeline.png)

##### Training policy network

In [30]:
from deeppavlov import train_model

gobot_config['train']['batch_size'] = 8 # set batch size
gobot_config['train']['max_batches'] = 200 # maximum number of training batches
gobot_config['train']['val_every_n_batches'] = 50 # evaluate on full 'valid' split each 50 batches
gobot_config['train']['log_every_n_batches'] = 50 # evaluate on 30 batches of 'train' split every 50 batches
gobot_config['train']['log_on_k_batches'] = 50

train_model(gobot_config);

2019-09-02 18:10:15.683 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-09-02 18:10:15.878 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-09-02 18:10:16.29 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
2019-09-02 18:10:16.344 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 98: [saving vocabulary to /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
W0902 18:10:17.340889 139952681506432 deprecation_wrapper.py:119] From /home/vimary/code-projects/Pilot/deeppavlov/core/models/tf_model.py:38: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0902 18:10:17.341

2019-09-02 18:10:19.498 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 251: [initializing `GoalOrientedBot` from scratch]
I0902 18:10:19.498898 139952681506432 network.py:251] [initializing `GoalOrientedBot` from scratch]
2019-09-02 18:11:40.632 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.0117
I0902 18:11:40.632941 139952681506432 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.0117
2019-09-02 18:11:40.633 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0902 18:11:40.633703 139952681506432 nn_trainer.py:166] Saving model
2019-09-02 18:11:40.634 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0902 18:11:40.634723 139952681506432 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
W0902 18:11:40.635666 139952681506432 deprecation_w

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.0117}, "time_spent": "0:01:22", "epochs_done": 0, "batches_seen": 0, "train_examples_seen": 0, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 400, "metrics": {"per_item_dialog_accuracy": 0.4774}, "time_spent": "0:03:01", "epochs_done": 0, "batches_seen": 50, "train_examples_seen": 400, "learning_rate": 0.003, "momentum": 0.95, "loss": 1.4143262553215026}}


2019-09-02 18:14:39.744 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 164: New best per_item_dialog_accuracy of 0.3861
I0902 18:14:39.744812 139952681506432 nn_trainer.py:164] New best per_item_dialog_accuracy of 0.3861
2019-09-02 18:14:39.745 INFO in 'deeppavlov.core.trainers.nn_trainer'['nn_trainer'] at line 166: Saving model
I0902 18:14:39.745602 139952681506432 nn_trainer.py:166] Saving model
2019-09-02 18:14:39.746 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 76: [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0902 18:14:39.746308 139952681506432 tf_model.py:76] [saving model to /home/vimary/code-projects/Pilot/examples/my_bot/model]
2019-09-02 18:14:39.812 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 764: [saving parameters to /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0902 18:14:39.812906 139952681506432 network.py:764] [saving parameters to /home/vimary/code-projects/Pilot/examp

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.3861}, "time_spent": "0:04:21", "epochs_done": 0, "batches_seen": 50, "train_examples_seen": 400, "impatience": 0, "patience_limit": 10}}
{"train": {"eval_examples_count": 400, "metrics": {"per_item_dialog_accuracy": 0.5425}, "time_spent": "0:06:02", "epochs_done": 0, "batches_seen": 100, "train_examples_seen": 800, "learning_rate": 0.003, "momentum": 0.95, "loss": 0.700360717177391}}


...

2019-09-02 18:23:44.74 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 248: [initializing `GoalOrientedBot` from saved]
I0902 18:23:44.074719 139952681506432 network.py:248] [initializing `GoalOrientedBot` from saved]
2019-09-02 18:23:44.75 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 770: [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0902 18:23:44.075809 139952681506432 network.py:770] [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
2019-09-02 18:23:44.79 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0902 18:23:44.079855 139952681506432 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]


{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4099}, "time_spent": "0:01:22"}}


2019-09-02 18:26:24.917 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
I0902 18:26:24.917217 139952681506432 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-09-02 18:26:24.918 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
W0902 18:26:24.918635 139952681506432 serializable.py:47] No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-02 18:26:24.919 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
I0902 18:26:24.919691 139952681506432 sqlite_database.py:66] Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-09-02 18:26:25.0 INFO

{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4115}, "time_spent": "0:01:19"}}


2019-09-02 18:26:25.631 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 248: [initializing `GoalOrientedBot` from saved]
I0902 18:26:25.631606 139952681506432 network.py:248] [initializing `GoalOrientedBot` from saved]
2019-09-02 18:26:25.632 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 770: [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
I0902 18:26:25.632694 139952681506432 network.py:770] [loading parameters from /home/vimary/code-projects/Pilot/examples/my_bot/model.json]
2019-09-02 18:26:25.635 INFO in 'deeppavlov.core.models.tf_model'['tf_model'] at line 52: [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]
I0902 18:26:25.635756 139952681506432 tf_model.py:52] [loading model from /home/vimary/code-projects/Pilot/examples/my_bot/model]


Training takes from 10 to 30 minutes depending on gpu/cpu.

See [config doc page](http://docs.deeppavlov.ai/en/master/intro/configuration.html) for advanced configuration of the training process.

##### Metric scores on valid&test

Calculating **accuracy** of trained bot: whether predicted system responses match true responses (full string match).

In [31]:
from deeppavlov import evaluate_model

evaluate_model(gobot_config);

2019-09-02 18:26:25.672 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
I0902 18:26:25.672325 139952681506432 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-trn.jsonlist]
2019-09-02 18:26:25.945 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
I0902 18:26:25.945016 139952681506432 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-val.jsonlist]
2019-09-02 18:26:26.13 INFO in 'deeppavlov.dataset_readers.dstc2_reader'['dstc2_reader'] at line 112: [loading dialogs from /home/vimary/code-projects/Pilot/examples/my_data/dstc2-tst.jsonlist]
I0902 18:26:26.013795 139952681506432 dstc2_reader.py:112] [loading dialogs from /home/vimary/code-projects/Pilot/

{"valid": {"eval_examples_count": 575, "metrics": {"per_item_dialog_accuracy": 0.4099}, "time_spent": "0:01:22"}}
{"test": {"eval_examples_count": 576, "metrics": {"per_item_dialog_accuracy": 0.4115}, "time_spent": "0:01:20"}}


With settings of `max_batches=200`, valid accuracy `~ 0.4` and test accuracy is `0.4`.

##### Chatting with bot

In [32]:
from deeppavlov import build_model

bot = build_model(gobot_config)

2019-09-02 18:29:07.670 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 112: [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
I0902 18:29:07.670491 139952681506432 simple_vocab.py:112] [loading vocabulary from /home/vimary/code-projects/Pilot/examples/my_bot/word.dict]
2019-09-02 18:29:07.672 WARNING in 'deeppavlov.core.models.serializable'['serializable'] at line 47: No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
W0902 18:29:07.672522 139952681506432 serializable.py:47] No load path is set for Sqlite3Database in 'infer' mode. Using save path instead
2019-09-02 18:29:07.673 INFO in 'deeppavlov.core.data.sqlite_database'['sqlite_database'] at line 66: Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
I0902 18:29:07.673603 139952681506432 sqlite_database.py:66] Loading database from /home/vimary/code-projects/Pilot/examples/my_bot/db.sqlite.
2019-09-02 18:29:07.770 IN

In [33]:
bot(['hi, want chinee food, can you suggest a place to go?'])

2019-09-02 18:29:08.395 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 505: Made api_call with {'food': 'chinese'}, got 16 results.
I0902 18:29:08.395405 139952681506432 network.py:505] Made api_call with {'food': 'chinese'}, got 16 results.


['The lucky star serves chinese food.']

In [34]:
bot(['ok, i want ther address'])

['Sure, the lucky star is on cambridge leisure park clifton way cherry hinton.']

In [35]:
bot(['now i want french food in the center of town'])

2019-09-02 18:29:08.490 INFO in 'deeppavlov.models.go_bot.network'['network'] at line 505: Made api_call with {'food': 'french', 'area': 'centre'}, got 1 results.
I0902 18:29:08.490886 139952681506432 network.py:505] Made api_call with {'food': 'french', 'area': 'centre'}, got 1 results.


['Cote is a nice place in the centre of town serving tasty french food.']

In [36]:
bot(['thanks, give me their address too'])

['Sure, cote is on bridge street city centre.']

In [37]:
bot(['and post code'])

['The post code of cote is c.b 2, 1 u.f.']

In [38]:
bot(['bye'])

['You are welcome!']

In [39]:
bot.reset()

In [40]:
bot(['hi, is there any cheap restaurant?'])

['What kind of food would you like?']